In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import torchvision.utils as vutils

In [2]:
from easydict import EasyDict as edict
opt = edict()
# data params
opt.dataroot = '/hd3/yekui/facades/'
opt.workers = 2

# network params
opt.input_nc = 3
opt.output_nc = 3
opt.ngf = 64
opt.ndf = 64
opt.netG = 'experiment/netG_epoch_19.pth'
opt.netD = 'experiment/netD_epoch_19.pth'

# training params
opt.adam = False
opt.cuda = True
opt.niter = 100     # number of epochs
opt.Diters = 25    # train the discriminator Diters times
opt.experiment = 'experiment'
opt.clamp_lower = -0.01
opt.clamp_upper =  0.01
opt.batch_size = 1
opt.lr = 0.0002
opt.beta1 = 0.8

In [3]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [14]:
# define generator
class generator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf):
        super(generator, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf     
        
        self.conv1 = nn.Conv2d(input_nc, ngf, 4, stride=2, padding=1, bias=False)      
        self.conv2 = nn.Conv2d(ngf, ngf*2, 4, stride=2, padding=1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(ngf*2)        
        self.conv3 = nn.Conv2d(ngf*2, ngf*4, 4, stride=2, padding=1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(ngf*4)        
        self.conv4 = nn.Conv2d(ngf*4, ngf*8, 4, stride=2, padding=1, bias=False)
        self.batchnorm4 = nn.BatchNorm2d(ngf*8)        
        self.conv5 = nn.Conv2d(ngf*8, ngf*8, 4, stride=2, padding=1, bias=False)
        self.batchnorm5 = nn.BatchNorm2d(ngf*8)        
        self.conv6 = nn.Conv2d(ngf*8, ngf*8, 4, stride=2, padding=1, bias=False)
        self.batchnorm6 = nn.BatchNorm2d(ngf*8)        
        self.conv7 = nn.Conv2d(ngf*8, ngf*8, 4, stride=2, padding=1, bias=False)
        self.batchnorm7 = nn.BatchNorm2d(ngf*8)        
        self.conv8 = nn.Conv2d(ngf*8, ngf*8, 4, stride=2, padding=1, bias=False)
        self.batchnorm8 = nn.BatchNorm2d(ngf*8)
        
        self.conv1d = nn.ConvTranspose2d(ngf*8*2,ngf*8,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm1d = nn.BatchNorm2d(ngf*8)        
        self.conv2d = nn.ConvTranspose2d(ngf*8*2,ngf*8,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm2d = nn.BatchNorm2d(ngf*8)        
        self.conv3d = nn.ConvTranspose2d(ngf*8*2,ngf*8,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm3d = nn.BatchNorm2d(ngf*8)        
        self.conv4d = nn.ConvTranspose2d(ngf*8*2,ngf*8,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm4d = nn.BatchNorm2d(ngf*8)        
        self.conv5d = nn.ConvTranspose2d(ngf*8*2,ngf*4,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm5d = nn.BatchNorm2d(ngf*4)        
        self.conv6d = nn.ConvTranspose2d(ngf*4*2,ngf*2,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm6d = nn.BatchNorm2d(ngf*2)        
        self.conv7d = nn.ConvTranspose2d(ngf*2*2,ngf,kernel_size=4, stride=2, padding=1, bias=False)
        self.batchnorm7d = nn.BatchNorm2d(ngf)        
        self.conv8d = nn.ConvTranspose2d(ngf*2,output_nc,kernel_size=4, stride=2, padding=1, bias=False)
        
    def forward(self, x):                                       # x : batch * 3  * 256 * 256                                                                
        x1 = self.conv1(x)                                      # x1: batch * 64 * 128 * 128
        x2 = self.batchnorm2(self.conv2(F.leaky_relu(x1, 0.2))) # x2: batch * 128 * 64 * 64
        x3 = self.batchnorm3(self.conv3(F.leaky_relu(x2, 0.2))) # x3: batch * 256 * 32 * 32
        x4 = self.batchnorm4(self.conv4(F.leaky_relu(x3, 0.2))) # x4: batch * 512 * 16 * 16
        x5 = self.batchnorm5(self.conv5(F.leaky_relu(x4, 0.2))) # x5: batch * 512 *  8 *  8
        x6 = self.batchnorm6(self.conv6(F.leaky_relu(x5, 0.2))) # x6: batch * 512 *  4 *  4
        x7 = self.batchnorm7(self.conv7(F.leaky_relu(x6, 0.2))) # x7: batch * 512 *  2 *  2
        x8 = self.batchnorm8(self.conv8(F.leaky_relu(x7, 0.2))) # x8: batch * 512 *  1 *  1
        
        noise = torch.randn(opt.batch_size, self.ngf * 8, 1, 1)
        noise = Variable(noise)
        d0 = torch.cat((noise, x8), 1)
        print(d0.size())
        d1_ = self.batchnorm1d(self.conv1d(F.relu(d0))) # d1_: batch * 512 *  2 *  2
        d1 = torch.cat((d1_, x7), 1)        # d1:  batch * 1024 *  2 *  2
        d2_ = self.batchnorm2d(self.conv2d(F.relu(d1))) # d2_: batch * 512 *  4 *  4
        d2 = torch.cat((d2_, x6), 1)        # d2:  batch * 1024 *  4 *  4
        d3_ = self.batchnorm3d(self.conv3d(F.relu(d2))) # d3_: batch * 512 *  8 *  8
        d3 = torch.cat((d3_, x5), 1)        # d3:  batch * 1024 *  8 *  8
        d4_ = self.batchnorm4d(self.conv4d(F.relu(d3))) # d4_: batch * 512 *  16 *  16
        d4 = torch.cat((d4_, x4), 1)        # d4:  batch * 1024 *  16 *  16
        d5_ = self.batchnorm5d(self.conv5d(F.relu(d4))) # d5_: batch * 256 *  32 *  32
        d5 = torch.cat((d5_, x3), 1)        # d5:  batch * 512 *  32 *  32
        d6_ = self.batchnorm6d(self.conv6d(F.relu(d5))) # d6_: batch * 128 *  64 *  64
        d6 = torch.cat((d6_, x2), 1)        # d6:  batch * 256 *  64 *  64
        d7_ = self.batchnorm7d(self.conv7d(F.relu(d6))) # d6_: batch * 64 *  128 *  128
        d7 = torch.cat((d7_, x1), 1)        # d7:  batch * 128 *  128 *  128
        d8 = self.conv8d(F.relu(d7))        # d8:  batch * 3   *  256 *  256
        
        out = F.tanh(d8)
        return out

input = torch.randn(opt.batch_size, 3, 256, 256)
input = Variable(input)
one = torch.FloatTensor(1,3,256,256).fill_(1)
netG = generator(opt.input_nc, opt.output_nc, opt.ngf)
netG.apply(weights_init)
res = netG(input)
print(res.size())
res.backward(one)



torch.Size([1, 1024, 1, 1])
torch.Size([1, 3, 256, 256])


In [15]:
import tflearn

Help on class BCELoss in module torch.nn.modules.loss:

class BCELoss(_WeighedLoss)
 |  Creates a criterion that measures the Binary Cross Entropy 
 |  between the target and the output:
 |  
 |  :math:`loss(o, t) = - 1/n \sum_i (t[i] * log(o[i]) + (1 - t[i]) * log(1 - o[i]))`
 |  
 |  or in the case of the weights argument being specified:
 |  
 |  :math:`loss(o, t) = - 1/n \sum_i weights[i] * (t[i] * log(o[i]) + (1 - t[i]) * log(1 - o[i]))`
 |  
 |  This is used for measuring the error of a reconstruction in for example 
 |  an auto-encoder. Note that the targets `t[i]` should be numbers between 0 and 1, 
 |  for instance, the output of an `nn.Sigmoid` layer.
 |  
 |  By default, the losses are averaged for each minibatch over observations 
 |  *as well as* over dimensions. However, if the field `sizeAverage` is set 
 |  to `False`, the losses are instead summed.
 |  
 |  Method resolution order:
 |      BCELoss
 |      _WeighedLoss
 |      _Loss
 |      torch.nn.modules.module.Modul